In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
import shutil

In [2]:
# 数组整形
def resize_arrays(A, B, fill_value=0):
    """调整数组形状一致,A为参考数组, B为待调整数组, fill_value为填充值"""
    # new_shape = (max(A.shape[0], B.shape[0]), max(A.shape[1], B.shape[1]))
    new_shape = (A.shape[0], A.shape[1])

    if A.shape != new_shape:
        if A.shape[0] < new_shape[0]:
            padding_rows = new_shape[0] - A.shape[0]
            padding = np.full((padding_rows, A.shape[1]), fill_value)
            A = np.vstack((A, padding))
        elif A.shape[0] > new_shape[0]:
            A = A[:new_shape[0], :]

        if A.shape[1] < new_shape[1]:
            pad_width = ((0, 0), (0, new_shape[1] - A.shape[1]))
            A = np.pad(A, pad_width, mode='constant', constant_values=fill_value)
        elif A.shape[1] > new_shape[1]:
            A = A[:, :new_shape[1]]
    
    if B.shape != new_shape:
        if B.shape[0] < new_shape[0]:
            padding_rows = new_shape[0] - B.shape[0]
            padding = np.full((padding_rows, B.shape[1]), fill_value)
            B = np.vstack((B, padding))
        elif B.shape[0] > new_shape[0]:
            B = B[:new_shape[0], :]

        if B.shape[1] < new_shape[1]:
            pad_width = ((0, 0), (0, new_shape[1] - B.shape[1]))
            B = np.pad(B, pad_width, mode='constant', constant_values=fill_value)
        elif B.shape[1] > new_shape[1]:
            B = B[:, :new_shape[1]]
    
    return A, B
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster
# 对齐栅格
def align_raster(in_raster,stander_raster):
    with arcpy.EnvManager(snapRaster=stander_raster, extent="MAXOF", cellSize=stander_raster, mask=stander_raster):
        in_raster = Raster(in_raster)
        stander_raster = Raster(stander_raster)
        output_raster = Con( in_raster>0, in_raster, stander_raster)
    return output_raster

In [3]:
in_raster_path = r'D:\ArcGISProjects\workspace\shbyq\MyProject.gdb\dy_dz_PolygonToRaster'
stander_raster_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM'

In [4]:
# 检查形状
in_raster_numpy = arcpy.RasterToNumPyArray(in_raster_path)
stander_raster_numpy = arcpy.RasterToNumPyArray(stander_raster_path)
print(in_raster_numpy.shape, stander_raster_numpy.shape)

(14812, 20053) (12881, 13343)


In [5]:
in_raster_check_numpy = resize_arrays(stander_raster_numpy,in_raster_numpy,0)

In [6]:
in_raster_check_numpy[0].shape,in_raster_check_numpy[1].shape

((12881, 13343), (12881, 13343))

In [8]:
# 输出整形后的栅格图
env.workspace = os.path.dirname(stander_raster_path)
raster_array = in_raster_check_numpy[1]
env.extent = "DEM"  # 指定输出栅格的范围
pred_result_raster = mask_raster(raster_array, "DEM",5)
pred_result_raster.save('DZ')

In [14]:
env.workspace = os.path.dirname(stander_raster_path)
result_align_raster = align_raster(in_raster_path,stander_raster_path)

In [17]:
result_align_raster.save("DL")